# Assignment 2

In this assigment, we will work with the *Adult* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/2/adult). Extract the data files into the subdirectory: `../05_src/data/adult/` (relative to `./05_src/`).

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../05_src/data/adult/`, then you can use the code below to load them.

In [24]:
import pandas as pd
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
    'native-country', 'income'
]
adult_dt = (pd.read_csv('/Users/Zhiyang/DSI/scaling_to_production/05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))


In [25]:
adult_dt.sample(n = 5)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
1719,38,Private,187069,HS-grad,9,Married-civ-spouse,Transport-moving,Husband,White,Male,0,0,45,United-States,0
13014,28,Private,176137,HS-grad,9,Never-married,Other-service,Own-child,White,Female,0,0,32,United-States,0
31840,55,Private,125000,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,1
7162,42,Federal-gov,284403,HS-grad,9,Divorced,Adm-clerical,Not-in-family,Black,Male,0,0,40,United-States,0
30521,27,State-gov,212232,Some-college,10,Married-civ-spouse,Exec-managerial,Husband,Black,Male,0,0,40,United-States,1


# Get X and Y

Create the features data frame and target data:

+ Create a dataframe `X` that holds the features (all columns that are not `income`).
+ Create a dataframe `Y` that holds the target data (`income`).
+ From `X` and `Y`, obtain the training and testing data sets:

    - Use a train-test split of 70-30%. 
    - Set the random state of the splitting function to 42.

In [26]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [27]:
X0 = adult_dt.drop('income', axis=1)
Y0 = adult_dt['income']
X0_train, X0_test, Y0_train, Y0_test = train_test_split(X0, Y0, test_size=0.3, random_state=42)


## Random States

Please comment: 

+ What is the [random state](https://scikit-learn.org/stable/glossary.html#term-random_state) of the [splitting function](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)? \
It's a process that we use the seed number and the random number generator to split the data into test and training set. 


+ Why is it [useful](https://en.wikipedia.org/wiki/Reproducibility)?\
The seed number can ensure the same split everytime we run the code.\
It's useful for consistency and reproducibility.

*(Comment here.)*\
Plz see above

# Preprocessing

Create a [Column Transformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) that treats the features as follows:

- Numerical variables

    * Apply [KNN-based imputation for completing missing values](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html):
        
        + Consider the 7 nearest neighbours.
        + Weight each neighbour by the inverse of its distance, causing closer neigbours to have more influence than more distant ones.
    * [Scale features using statistics that are robust to outliers](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html#sklearn.preprocessing.RobustScaler).

- Categorical variables: 
    
    * Apply a [simple imputation strategy](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer):

        + Use the most frequent value to complete missing values, also called the *mode*.

    * Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html):
        
        + Handle unknown labels if they exist.
        + Drop one column for binary variables.
    
    
The column transformer should look like this:

![](./images/assignment_2__column_transformer.png)

In [28]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import RobustScaler
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer

In [29]:
num_cols = ['age','fnlwgt','education-num','capital-gain','capital-loss','hours-per-week']

cat_cols = ['workclass', 'education', 'marital-status','occupation','relationship','race','sex','native-country']

numerical_pipeline = Pipeline([
    ('KNNImputer', KNNImputer(n_neighbors=7, weights='distance')),
    ('RobustScaler', RobustScaler())
])

categorical_pipeline = Pipeline([
    ('SimpleImputer', SimpleImputer(strategy='most_frequent')),
    ('OneHotEncoder', OneHotEncoder(handle_unknown='ignore', drop='if_binary'))
])


transformer = ColumnTransformer([
    ('num_transforms', numerical_pipeline, num_cols),
    ('cat_transforms', categorical_pipeline, cat_cols)
])


In [30]:
transformer

ColumnTransformer(transformers=[('num_transforms',
                                 Pipeline(steps=[('KNNImputer',
                                                  KNNImputer(n_neighbors=7,
                                                             weights='distance')),
                                                 ('RobustScaler',
                                                  RobustScaler())]),
                                 ['age', 'fnlwgt', 'education-num',
                                  'capital-gain', 'capital-loss',
                                  'hours-per-week']),
                                ('cat_transforms',
                                 Pipeline(steps=[('SimpleImputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('OneHotEncoder',
                                                  OneHotEncoder(drop='if_binary',
                                                                handle_unknown='ignore'))]),
                                 ['workclass', 'education', 'marital-status',
                                  'occupation', 'relationship', 'race', 'sex',
                                  'native-country'])])

## Model Pipeline

Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `classifier` and assign a [`RandomForestClassifier()`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) to it.

The pipeline looks like this:

![](./images/assignment_2__pipeline.png)

In [31]:
from sklearn.ensemble import RandomForestClassifier

model_pipeline = Pipeline([
    ('preprocessing', transformer), 
    ('classifier', RandomForestClassifier())  
])

In [32]:
model_pipeline

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_transforms',
                                                  Pipeline(steps=[('KNNImputer',
                                                                   KNNImputer(n_neighbors=7,
                                                                              weights='distance')),
                                                                  ('RobustScaler',
                                                                   RobustScaler())]),
                                                  ['age', 'fnlwgt',
                                                   'education-num',
                                                   'capital-gain',
                                                   'capital-loss',
                                                   'hours-per-week']),
                                                 ('cat_transforms',
                                                  Pipeline(steps=[('SimpleImputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('OneHotEncoder',
                                                                   OneHotEncoder(drop='if_binary',
                                                                                 handle_unknown='ignore'))]),
                                                  ['workclass', 'education',
                                                   'marital-status',
                                                   'occupation', 'relationship',
                                                   'race', 'sex',
                                                   'native-country'])])),
                ('classifier', RandomForestClassifier())])

# Cross-Validation

Evaluate the model pipeline using [`cross_validate()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html):

+ Measure the following [preformance metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values): negative log loss, ROC AUC, accuracy, and balanced accuracy.
+ Report the training and validation results. 
+ Use five folds.


In [33]:
from sklearn.model_selection import cross_validate

scoring = ['neg_log_loss', 'roc_auc', 'accuracy', 'balanced_accuracy']

results = cross_validate(model_pipeline, X0_train, Y0_train, cv=5, scoring=scoring, return_train_score=True)


/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:241: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:241: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [34]:
pd.DataFrame(results)

,fit_time,score_time,test_neg_log_loss,train_neg_log_loss,test_roc_auc,train_roc_auc,test_accuracy,train_accuracy,test_balanced_accuracy,train_balanced_accuracy
0,12.067456,0.277165,-0.358713,-0.081234,0.903214,1.0,0.849528,1.000000,0.776219,1.000000
1,10.765859,0.285735,-0.361815,-0.081856,0.901725,1.0,0.846677,0.999945,0.766961,0.999887
2,11.859992,0.471458,-0.382698,-0.081395,0.901615,1.0,0.851470,1.000000,0.773048,1.000000
3,9.264405,0.219709,-0.371704,-0.081841,0.906167,1.0,0.857613,0.999890,0.781103,0.999774
4,10.122444,0.227060,-0.394640,-0.081608,0.902079,1.0,0.858929,1.000000,0.783510,1.000000


Display the fold-level results as a pandas data frame and sorted by negative log loss of the test (validation) set.

In [35]:
df_results = pd.DataFrame(results)
df_results['fold'] = range(1, len(df_results) + 1)

df_results.sort_values(by='test_neg_log_loss', ascending=False)

,fit_time,score_time,test_neg_log_loss,train_neg_log_loss,test_roc_auc,train_roc_auc,test_accuracy,train_accuracy,test_balanced_accuracy,train_balanced_accuracy,fold
0,12.067456,0.277165,-0.358713,-0.081234,0.903214,1.0,0.849528,1.000000,0.776219,1.000000,1
1,10.765859,0.285735,-0.361815,-0.081856,0.901725,1.0,0.846677,0.999945,0.766961,0.999887,2
3,9.264405,0.219709,-0.371704,-0.081841,0.906167,1.0,0.857613,0.999890,0.781103,0.999774,4
2,11.859992,0.471458,-0.382698,-0.081395,0.901615,1.0,0.851470,1.000000,0.773048,1.000000,3
4,10.122444,0.227060,-0.394640,-0.081608,0.902079,1.0,0.858929,1.000000,0.783510,1.000000,5


Calculate the mean of each metric. 

In [36]:
df_results.mean()

fit_time                   10.816031
score_time                  0.296225
test_neg_log_loss          -0.373914
train_neg_log_loss         -0.081587
test_roc_auc                0.902960
train_roc_auc               1.000000
test_accuracy               0.852844
train_accuracy              0.999967
test_balanced_accuracy      0.776168
train_balanced_accuracy     0.999932
fold                        3.000000
dtype: float64

Calculate the same performance metrics (negative log loss, ROC AUC, accuracy, and balanced accuracy) using the testing data `X_test` and `Y_test`. Display results as a dictionary.

*Tip*: both, `roc_auc()` and `neg_log_loss()` will require prediction scores from `pipe.predict_proba()`. However, for `roc_auc()` you should only pass the last column `Y_pred_proba[:, 1]`. Use `Y_pred_proba` with `neg_log_loss()`.

In [37]:
from sklearn.metrics import log_loss, roc_auc_score, accuracy_score, balanced_accuracy_score

In [38]:
model_pipeline.fit(X0_train, Y0_train)

# Predict
Y_pred = model_pipeline.predict(X0_test)
Y_pred_proba = model_pipeline.predict_proba(X0_test)


performance_metrics = {
    'neg_log_loss': -log_loss(Y0_test, Y_pred_proba),
    'roc_auc': roc_auc_score(Y0_test, Y_pred_proba[:, 1]),
    'accuracy': accuracy_score(Y0_test, Y_pred),
    'balanced_accuracy': balanced_accuracy_score(Y0_test, Y_pred)
}

performance_metrics

{'neg_log_loss': -0.37511217119540663,
 'roc_auc': 0.9016349610193573,
 'accuracy': 0.8525949431876344,
 'balanced_accuracy': 0.7725923967892634}

# Target Recoding

In the first code chunk of this document, we loaded the data and immediately recoded the target variable `income`. Why is this [convenient](https://scikit-learn.org/stable/modules/model_evaluation.html#binary-case)?

The specific line was:

```
adult_dt = (pd.read_csv('../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))
```

(Answer here.)

We recoded the target variable income right away so that we won't forget to recode it later and simplify the process, and the values would be saved consistently.\
Also, income is a binary variable, we clearly define it to avoid misunderstanding later, so that we could see the probability of income >50k.


## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Evaluation of model pipeline |Model pipeline was evaluated correctly.|Model pipeline was not evaluated correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Becker,Barry and Kohavi,Ronny. (1996). Adult. UCI Machine Learning Repository. https://doi.org/10.24432/C5XW20.